In [0]:
import collections
import datetime
import numpy as np
import pandas as pd
import subprocess
import tensorflow as tf
from tensorflow import keras

In [0]:
B = 32

In [2]:
subprocess.run(["rm", "-rf", "./logs/"])


CompletedProcess(args=['rm', '-rf', './logs/'], returncode=0)

In [0]:
def count_words(title_list):
    c = collections.Counter(
        word for title in title_list for word in title.split())
    return {word: i + 1 for i, (word, cnt) in enumerate(c.most_common()) if cnt > 1}


def label_encoder(title, word_dict):
    return [word_dict.get(word, 0) for word in title.split()]


In [0]:
train_df = pd.read_table('/content/drive/My Drive/Colab Notebooks/train.txt')
valid_df = pd.read_table('/content/drive/My Drive/Colab Notebooks/valid.txt')

In [0]:
word_dict = count_words(train_df['TITLE'])
num_words = len(word_dict) + 1  # word not in dict is labeled as 0


In [0]:
# label encode and padding with 0
train_X = keras.preprocessing.sequence.pad_sequences(
    train_df['TITLE'].map(lambda x: label_encoder(x, word_dict)))
valid_X = keras.preprocessing.sequence.pad_sequences(
    valid_df['TITLE'].map(lambda x: label_encoder(x, word_dict)))


In [0]:
train_y = train_df['CATEGORY'].map({'b': 0, 't': 1, 'e': 2, 'm': 3}).values
valid_y = valid_df['CATEGORY'].map({'b': 0, 't': 1, 'e': 2, 'm': 3}).values


In [0]:
model = keras.Sequential([
    keras.layers.Embedding(num_words+1, 300, mask_zero=True),
    keras.layers.SimpleRNN(50),
    keras.layers.Dense(4, activation='relu')
])

In [0]:
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

In [0]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)


In [13]:
model.fit(train_X, train_y, batch_size=B, epochs=10, validation_data=(valid_X, valid_y),
          callbacks=[tensorboard_callback])


Epoch 1/10
334/334 [==============================] - 98s 294ms/step - loss: 1.1295 - accuracy: 0.5253 - val_loss: 0.9887 - val_accuracy: 0.6267
Epoch 2/10
334/334 [==============================] - 96s 288ms/step - loss: 0.9497 - accuracy: 0.6505 - val_loss: 1.0722 - val_accuracy: 0.5667
Epoch 3/10
334/334 [==============================] - 99s 295ms/step - loss: 0.8487 - accuracy: 0.7009 - val_loss: 0.8103 - val_accuracy: 0.7294
Epoch 4/10
334/334 [==============================] - 97s 291ms/step - loss: 0.7792 - accuracy: 0.7259 - val_loss: 0.8344 - val_accuracy: 0.7076
Epoch 5/10
334/334 [==============================] - 97s 291ms/step - loss: 0.7057 - accuracy: 0.7492 - val_loss: 0.6573 - val_accuracy: 0.7714
Epoch 6/10
334/334 [==============================] - 97s 290ms/step - loss: 0.6429 - accuracy: 0.7711 - val_loss: 0.6141 - val_accuracy: 0.7939
Epoch 7/10
334/334 [==============================] - 96s 288ms/step - loss: 0.5892 - accuracy: 0.7900 - val_loss: 0.5754 - val_ac